In [2]:
import json
import yaml
import fitz

In [3]:
from doctr.models import ocr_predictor
model = ocr_predictor(det_arch="db_resnet50",reco_arch="crnn_vgg16_bn",pretrained=True)

c:\ProgramData\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
from doctr.io import DocumentFile
pdf_doc = DocumentFile.from_pdf('../../Downloads/021813s000_SumR.pdf')

In [5]:
result = model(pdf_doc)

In [6]:
output = result.export()

In [18]:
# import json
# with open('../../Downloads/DOC_TR_OUT1.json','w') as file:
#     file.write(json.dumps(output,indent=1))


In [7]:
with open('../../Downloads/DOC_TR_OUT1.json','r') as file:
    res_json  = json.load(file)

In [21]:
# len(res_json)

1

In [25]:
# no_of_paegs = len(res_json['pages'])

In [28]:
# page_idx_1  = res_json['pages'][1]

In [41]:
# for page in res_json['pages']:
#     print(page['page_idx'])
#     print(page['dimensions'])
#     print(page['blocks'])
#     print(page['blocks'])
#     print("no of blocks %d" %(len(page['blocks'])))
#     print(page['blocks'][0])
#     print(page['blocks'][0]['lines'])
#     print("no of lines %d" %(len(page['blocks'][0]['lines'])))
#     print(page['blocks'][0]['lines'][0])
#     print(page['blocks'][0]['lines'][0]['words'])
#     print("no of words %d" %(len(page['blocks'][0]['lines'][0]['words'])))
#     print(page['blocks'][0]['lines'][0]['words'][0])
#     break

0
[1584, 1224]
[{'geometry': [[0.173828125, 0.0693359375], [0.86328125, 0.09375]], 'lines': [{'geometry': [[0.173828125, 0.0693359375], [0.86328125, 0.09375]], 'words': [{'value': 'CENTER', 'confidence': 0.9995423555374146, 'geometry': [[0.173828125, 0.0712890625], [0.322265625, 0.091796875]]}, {'value': 'FOR', 'confidence': 0.9999570846557617, 'geometry': [[0.330078125, 0.0693359375], [0.4111328125, 0.09375]]}, {'value': 'DRUG', 'confidence': 0.9995051622390747, 'geometry': [[0.4169921875, 0.0712890625], [0.5283203125, 0.0927734375]]}, {'value': 'EVALUATION', 'confidence': 0.9956958889961243, 'geometry': [[0.5283203125, 0.0712890625], [0.7763671875, 0.091796875]]}, {'value': 'AND', 'confidence': 0.9962904453277588, 'geometry': [[0.7841796875, 0.0712890625], [0.86328125, 0.0927734375]]}]}], 'artefacts': []}, {'geometry': [[0.41796875, 0.103515625], [0.6171875, 0.1240234375]], 'lines': [{'geometry': [[0.41796875, 0.103515625], [0.6171875, 0.1240234375]], 'words': [{'value': 'RESEARCH', 

In [8]:
page_coordinates = {}
for page in res_json['pages']:
    page_idx = page['page_idx']
    for block_idx,block in enumerate(page['blocks']):
        block_idx = block_idx
        for line_idx,lines in enumerate(block['lines']):
            line_idx = line_idx
            for wrd_idx,word in enumerate(lines['words']):
                wrd_idx = wrd_idx
                wrd_value = word['value']
                wrd_geometry  = word['geometry']
                page_coordinates[str((page_idx,block_idx,line_idx,wrd_idx))] = (wrd_geometry,wrd_value)

            


In [9]:
import yaml
with open('../../Downloads/page_coordinates.yaml','w') as file:
    yaml.safe_dump(page_coordinates,file)

In [10]:
import math
def scale_coords_to_page_dim(coords,page_dims):
    len_x = page_dims[0]
    len_y = page_dims[1]

    (x_min,y_min) = coords[0]
    (x_max,y_max) = coords[1]

    scaled_x_min  = math.floor(x_min * len_x)
    scaled_y_min = math.floor(y_min * len_y)
    scaled_x_max = math.ceil(x_max * len_x)
    scaled_y_max = math.ceil(y_max * len_y)

    return (scaled_x_min,scaled_y_min,scaled_x_max,scaled_y_max)


    

In [11]:
scaled_page_coordinates = {}
for page in res_json['pages']:
    page_idx = page['page_idx']
    dimensions = page['dimensions']
    for block_idx,block in enumerate(page['blocks']):
        block_idx = block_idx
        for line_idx,lines in enumerate(block['lines']):
            line_idx = line_idx
            for wrd_idx,word in enumerate(lines['words']):
                wrd_idx = wrd_idx
                wrd_value = word['value']
                wrd_geometry  = word['geometry']
                scaled_coords = scale_coords_to_page_dim(wrd_geometry,dimensions)
                scaled_page_coordinates[str((page_idx,block_idx,line_idx,wrd_idx))] = (scaled_coords,wrd_value)

In [12]:

import yaml
with open('../../Downloads/scaled_page_coordinates.yaml','r') as file:
    file = yaml.safe_load(file)

In [13]:
import PIL
from PIL import ImageDraw
import matplotlib.pyplot as plt

def draw_bounds(image,bounds):
    draw = ImageDraw.Draw(image)
    for b in bounds:
        p0, p1, p2, p3 = [b[0],b[2]], [b[1],b[2]], \
                         [b[1],b[3]], [b[0],b[3]]
        draw.line([*p0,*p1,*p2,*p3,*p0], fill='blue', width=2)
    return image

In [78]:
file = fitz.open('../../Downloads/021813s000_SumR.pdf')

In [47]:
for page in file.pages():
    print(page)
    x_min = math.floor(0.1738281250 * page.rect.width)
    y_min = math.floor(0.0712890625 * page.rect.height)
    x_max = math.ceil(0.322265625 * page.rect.width)
    y_max = math.ceil(0.091796875 * page.rect.height)
    r = [x_min,y_min,x_max,y_max]
    shape = page.new_shape()
    # shape.insert_textbox(r)
    # shape.draw_rect((r[0],r[1]),(r[2],r[3]))
    shape.draw_rect(r)
    shape.finish(color=(0, 0, 1), fill=None)
    
    shape.commit()

    # pix = page.get_pixmap()
    # pix.save(f'../../Downloads/page-%i.png' %(page.number))
file.save("draw_rect_6.pdf")

page 0 of ../../Downloads/021813s000_SumR.pdf
page 1 of ../../Downloads/021813s000_SumR.pdf
page 2 of ../../Downloads/021813s000_SumR.pdf
page 3 of ../../Downloads/021813s000_SumR.pdf
page 4 of ../../Downloads/021813s000_SumR.pdf
page 5 of ../../Downloads/021813s000_SumR.pdf
page 6 of ../../Downloads/021813s000_SumR.pdf
page 7 of ../../Downloads/021813s000_SumR.pdf
page 8 of ../../Downloads/021813s000_SumR.pdf
page 9 of ../../Downloads/021813s000_SumR.pdf
page 10 of ../../Downloads/021813s000_SumR.pdf
page 11 of ../../Downloads/021813s000_SumR.pdf
page 12 of ../../Downloads/021813s000_SumR.pdf


In [48]:
import yaml
with open('../../Downloads/scaled_page_coordinates.yaml','r') as file:
    file_yaml = yaml.safe_load(file)

In [59]:
file_yaml['(0, 0, 0, 0)']

[[275, 87, 511, 113], 'CENTER']

In [58]:
file_yaml['(0, 0, 0, 0)']

[[275, 87, 511, 113], 'CENTER']

In [68]:
new_keys = [eval(i)[0] for i in list(file_yaml.keys())]

In [69]:
len(new_keys)

2893

In [72]:
coords = [li[0] for li in list(file_yaml.values())]

In [73]:
len(coords)

2893

In [76]:
new_set_cords = tuple(zip(new_keys,coords))

In [82]:
for page in file.pages():
    for coord in new_set_cords:
        if page.number == coord[0]:
            print(coord[1])
            x_min = math.floor(coord[1][0] * page.rect.width)
            y_min = math.floor(coord[1][1] * page.rect.height)
            x_max = math.ceil(coord[1][2] * page.rect.width)
            y_max = math.ceil(coord[1][3] * page.rect.height)
            r = [x_min,y_min,x_max,y_max]
            shape = page.new_shape()
            shape.draw_rect(r)
            shape.finish(color=(0, 0, 1), fill=None)
            shape.commit()

file.save("draw_rect_7.pdf")